# Decentralized Data Distribution Network Setup

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

In [ ]:
fablib.list_sites();

In [ ]:
# Optional list of fields to display.
# fields=None desplays all fields.
fields=['name','cores_available','ram_available','disk_available','nic_basic_available']

In [ ]:
output_table = fablib.list_sites(fields=fields)

In [ ]:
slice_name="project-" + fablib.get_bastion_username()

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

In [ ]:
while True:
    ne_site_name = "RUTG"
    if ( (fablib.resources.get_core_available(ne_site_name) > 8) and
        (fablib.resources.get_component_available(ne_site_name, 'SharedNIC-ConnectX-6') > 8) ):
        fablib.show_site(ne_site_name)
        break
    else:
        print(f"Not enough resources available at: {ne_site_name}")
        break

while True:
    se_site_name = "GATECH"
    if ( (fablib.resources.get_core_available(se_site_name) > 8) and
        (fablib.resources.get_component_available(se_site_name, 'SharedNIC-ConnectX-6') > 8) ):
        fablib.show_site(se_site_name)
        break
    else:
        print(f"Not enough resources available at: {se_site_name}")
        break
        
while True:
    sw_site_name = "UTAH"
    if ( (fablib.resources.get_core_available(sw_site_name) > 8) and
        (fablib.resources.get_component_available(sw_site_name, 'SharedNIC-ConnectX-6') > 8) ):
        fablib.show_site(sw_site_name)
        break
    else:
        print(f"Not enough resources available at: {sw_site_name}")
        break

while True:
    sink_site_name = "DALL"
    if ( (fablib.resources.get_core_available(sink_site_name) > 8) and
        (fablib.resources.get_component_available(sink_site_name, 'SharedNIC-ConnectX-6') > 8) ):
        fablib.show_site(sink_site_name)
        break
    else:
        print(f"Not enough resources available at: {sink_site_name}")
        break


In [ ]:
# this cell sets up sink nodes
sink_rtr = slice.add_node(name='sink_router', site=sink_site_name, cores=2, ram=4, disk=10, image='default_ubuntu_20')
sink_rtr.add_fabnet()


sink_node = slice.add_node(name='sink', site=sink_site_name, cores=2, ram=4, disk=10, image='default_ubuntu_20')
sink_node.add_fabnet()


In [ ]:
# this cell sets up the NE nodes
NE_1 = slice.add_node(name='NE_1', site=ne_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
NE_2 = slice.add_node(name='NE_2', site=ne_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
NE_3 = slice.add_node(name='NE_3', site=ne_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
NE_4 = slice.add_node(name='NE_4', site=ne_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')

NE_1.add_fabnet()
NE_2.add_fabnet()
NE_3.add_fabnet()
NE_4.add_fabnet()

In [ ]:
# this cell sets up the SE nodes
SE_1 = slice.add_node(name='SE_1', site=se_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
SE_2 = slice.add_node(name='SE_2', site=se_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
SE_3 = slice.add_node(name='SE_3', site=se_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
SE_4 = slice.add_node(name='SE_4', site=se_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')

SE_1.add_fabnet()
SE_2.add_fabnet()
SE_3.add_fabnet()
SE_4.add_fabnet()

In [ ]:
# this cell sets up the SW nodes
SW_1 = slice.add_node(name='SW_1', site=sw_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
SW_2 = slice.add_node(name='SW_2', site=sw_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
SW_3 = slice.add_node(name='SW_3', site=sw_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')
SW_4 = slice.add_node(name='SW_4', site=sw_site_name, cores=1, ram=2, disk=5, image='default_ubuntu_20')

SW_1.add_fabnet()
SW_2.add_fabnet()
SW_3.add_fabnet()
SW_4.add_fabnet()

In [ ]:
slice.submit()

In [ ]:
slice.show();

In [ ]:
slice.list_nodes();

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
packages = ['net-tools', 'iperf3', 'moreutils']

for n in slice.get_nodes():
    pkg = " ".join(packages)
    n.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)


In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

In [ ]:
slice.update()
_ = slice.show()

# When complete with the project

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
slice.update()
_ = slice.show()